In [4]:
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import warnings


# Assuming spacex_df is a DataFrame containing the SpaceX data
# Replace with the actual path to your dataset
spacex_df =  pd.read_csv("C:/Users/micho/OneDrive/Desktop/PROJECTS HOUSE/spacex.csv")

# Extract unique launch sites
launch_sites = spacex_df['Launch Site'].unique()

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center'}),
    
    # Dropdown for selecting launch site
    dcc.Dropdown(id='site-dropdown',
                 options=[{'label': 'All Sites', 'value': 'ALL'}] + 
                         [{'label': site, 'value': site} for site in launch_sites],
                 value='ALL',
                 placeholder="Select a Launch Site here",
                 searchable=True),
    
    # Pie chart for success rate
    dcc.Graph(id='success-pie-chart'),
    
    # Range slider for payload selection
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={i: str(i) for i in range(0, 10001, 1000)},
                    value=[0, 10000]),
    
    # Scatter plot for payload vs. success
    dcc.Graph(id='success-payload-scatter-chart')
])

# Add Callback for Pie Chart
@app.callback(Output('success-pie-chart', 'figure'),
              Input('site-dropdown', 'value'))
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        filtered_df = spacex_df
        title = 'Total Success Launches by Site'
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        title = f'Total Success Launches for Site {selected_site}'
    
    fig = px.pie(filtered_df, 
                 names='class', 
                 title=title, 
                 hole=0.3)
    return fig

# Add Callback for Scatter Plot
@app.callback(Output('success-payload-scatter-chart', 'figure'),
              [Input('site-dropdown', 'value'),
               Input('payload-slider', 'value')])
def update_scatter_plot(selected_site, payload_range):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', 
                     color='Booster Version',
                     title='Payload vs. Success for All Sites' if selected_site == 'ALL' else f'Payload vs. Success for Site {selected_site}')
    return fig

# Run the App on a Different Port
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)  # Changed port to 8051
